# Gemini Deep Research Agent - Interactive Notebook

This notebook demonstrates Google's Deep Research Agent, which autonomously plans, executes, and synthesizes multi-step research tasks.

**Preview Notice:** The Deep Research Agent is free during preview. Google Search tool calls are free until January 5th, 2026.

## Setup

First, install the required package and set your API key.

In [ ]:
# Install the Google GenAI SDK
!pip install -q google-genai

In [ ]:
import os
from google import genai

# Set your API key (get one at https://aistudio.google.com/apikey)
# Option 1: Set directly (not recommended for sharing)
# os.environ['GOOGLE_API_KEY'] = 'your_key_here'

# Option 2: Use Colab secrets or environment variable
# The client will automatically use GOOGLE_API_KEY from environment

client = genai.Client()
print("Client initialized successfully!")

## Configuration

In [ ]:
# Agent configuration
DEEP_RESEARCH_AGENT = "deep-research-pro-preview-12-2025"
SUMMARY_MODEL = "gemini-2.5-flash"
FOLLOWUP_MODEL = "gemini-3-pro-preview"

## Example 1: Basic Research with Streaming

The Deep Research Agent works asynchronously. We use `background=True` and `stream=True` to watch its progress in real-time.

In [ ]:
def run_research(query: str, file_store: str = None) -> dict:
    """
    Run a deep research task with streaming output.
    
    Args:
        query: The research question or task
        file_store: Optional file store name for private document search
    
    Returns:
        dict with 'text' (final report), 'thoughts' (reasoning), and 'interaction_id'
    """
    # Configure tools
    tools = None
    if file_store:
        tools = [{"type": "file_search", "file_search_store_names": [file_store]}]
    
    # Start the research stream
    stream = client.interactions.create(
        input=query,
        agent=DEEP_RESEARCH_AGENT,
        background=True,
        stream=True,
        tools=tools,
        agent_config={"type": "deep-research", "thinking_summaries": "auto"}
    )
    
    full_text = ""
    thoughts = []
    interaction_id = None
    
    print("🔬 Research in progress...\n")
    print("=" * 50)
    
    for chunk in stream:
        # Capture interaction ID for follow-ups
        if chunk.event_type == "interaction.start":
            interaction_id = chunk.interaction.id
            print(f"📋 Interaction ID: {interaction_id}\n")
        
        # Handle content
        if chunk.event_type == "content.delta":
            if chunk.delta.type == "text":
                full_text += chunk.delta.text
                print(chunk.delta.text, end="", flush=True)
            elif chunk.delta.type == "thought_summary":
                thought = chunk.delta.content.text
                thoughts.append(thought)
                print(f"\n💭 {thought}\n", flush=True)
        
        # Completion
        if chunk.event_type == "interaction.complete":
            print("\n" + "=" * 50)
            print("✅ Research complete!")
    
    return {
        "text": full_text,
        "thoughts": thoughts,
        "interaction_id": interaction_id
    }

In [ ]:
# Run your research query
# This will take several minutes - the agent is doing real research!

result = run_research("Research the history and evolution of Google TPUs")

## Example 2: View the Agent's Reasoning

The `thoughts` captured during streaming show the agent's internal reasoning process.

In [ ]:
print("🧠 Agent's Reasoning Process:\n")
for i, thought in enumerate(result['thoughts'], 1):
    print(f"{i}. {thought}\n")

## Example 3: Follow-up Questions

You can ask follow-up questions about completed research without re-running the entire process.

In [ ]:
def ask_followup(interaction_id: str, question: str) -> str:
    """
    Ask a follow-up question about a completed research session.
    
    Args:
        interaction_id: The ID from the original research
        question: Your follow-up question
    
    Returns:
        The agent's response
    """
    response = client.interactions.create(
        input=question,
        model=FOLLOWUP_MODEL,
        previous_interaction_id=interaction_id
    )
    return response.outputs[-1].text

In [ ]:
# Ask a follow-up (only works if you have an interaction_id from above)
if result.get('interaction_id'):
    followup = ask_followup(
        result['interaction_id'],
        "What are the key differences between TPU v4 and v5?"
    )
    print(followup)
else:
    print("No interaction ID available. Run the research cell first.")

## Example 4: Structured Output with Formatting Instructions

You can steer the agent's output format by providing specific instructions in your prompt.

In [ ]:
structured_query = """
Research the competitive landscape of EV batteries.

Format the output as a technical report with the following structure:
1. Executive Summary (3-4 sentences)
2. Key Players (include a comparison table with company, chemistry type, and capacity)
3. Recent Developments (2024-2025)
4. Supply Chain Considerations
5. Outlook
"""

# Uncomment to run (takes several minutes)
# structured_result = run_research(structured_query)

## Example 5: Research with File Search (Your Own Documents)

If you have a File Store set up in Google AI Studio, you can include your private documents in the research.

In [ ]:
# First, create a File Store in Google AI Studio and upload your documents
# Then use it like this:

# file_search_result = run_research(
#     query="Compare our Q3 report against public market analysis",
#     file_store="fileSearchStores/my-company-docs"
# )

## Example 6: Polling Instead of Streaming

If you prefer not to stream, you can poll for results instead.

In [ ]:
import time

def run_research_polling(query: str) -> str:
    """
    Run research with polling instead of streaming.
    """
    # Start the research
    interaction = client.interactions.create(
        input=query,
        agent=DEEP_RESEARCH_AGENT,
        background=True
    )
    
    print(f"Research started: {interaction.id}")
    
    # Poll for completion
    while True:
        interaction = client.interactions.get(interaction.id)
        
        if interaction.status == "completed":
            print("\n✅ Complete!")
            return interaction.outputs[-1].text
        elif interaction.status == "failed":
            raise Exception(f"Research failed: {interaction.error}")
        
        print(".", end="", flush=True)
        time.sleep(10)

# Uncomment to run
# polling_result = run_research_polling("What are the latest developments in quantum computing?")
# print(polling_result)

## Generate Summary with Gemini Flash

Use a faster model to generate titles and summaries for cataloging.

In [ ]:
def generate_summary(query: str, result_text: str) -> tuple:
    """
    Generate a title and summary for a research result.
    
    Returns:
        tuple of (title, summary)
    """
    # Generate summary
    summary_prompt = f"Summarize this research in under 160 characters. Task: {query}\nResult: {result_text[:2000]}"
    summary_response = client.models.generate_content(
        model=SUMMARY_MODEL,
        contents=summary_prompt
    )
    summary = summary_response.text.strip()
    
    # Generate title
    title_prompt = f"Give a 3-5 word title for this research: {query}"
    title_response = client.models.generate_content(
        model=SUMMARY_MODEL,
        contents=title_prompt
    )
    title = title_response.text.strip().replace('"', '')
    
    return title, summary

In [ ]:
# Generate summary for your research
if result.get('text'):
    title, summary = generate_summary("Research the history of Google TPUs", result['text'])
    print(f"📌 Title: {title}")
    print(f"📝 Summary: {summary}")

## Save Research to JSON

Persist your research for future reference.

In [ ]:
import json
from datetime import datetime
import uuid

def save_research(query: str, result: dict, title: str = None, summary: str = None):
    """
    Save research to a JSON file.
    """
    session_id = str(uuid.uuid4())
    
    data = {
        "id": session_id,
        "interaction_id": result.get('interaction_id'),
        "timestamp": datetime.now().isoformat(),
        "title": title or "Untitled Research",
        "summary": summary or query[:157] + "...",
        "prompt": query,
        "response": result['text'],
        "thoughts": result['thoughts']
    }
    
    filename = f"research_{session_id[:8]}.json"
    with open(filename, 'w') as f:
        json.dump(data, f, indent=2)
    
    print(f"💾 Saved to {filename}")
    return filename

In [ ]:
# Save your research
if result.get('text'):
    filename = save_research(
        "Research the history of Google TPUs",
        result,
        title=title if 'title' in dir() else None,
        summary=summary if 'summary' in dir() else None
    )

---

## Quick Reference

| Feature | Code |
|---------|------|
| Basic research | `run_research("your query")` |
| With file search | `run_research("query", file_store="fileSearchStores/name")` |
| Follow-up question | `ask_followup(interaction_id, "question")` |
| Generate summary | `generate_summary(query, result_text)` |
| Save to JSON | `save_research(query, result)` |

**Models Used:**
- Deep Research: `deep-research-pro-preview-12-2025`
- Summaries: `gemini-2.5-flash`
- Follow-ups: `gemini-3-pro-preview`

**Links:**
- [Deep Research Documentation](https://ai.google.dev/gemini-api/docs/deep-research)
- [Interactions API](https://ai.google.dev/gemini-api/docs/interactions)
- [Get API Key](https://aistudio.google.com/apikey)